Let's export the trained model to HuggingFace Hub in safetensors formats for compatibility with downstream inference engines. First, we'll define some variables.

In [5]:
model_name = "UltraZoom-2X"
checkpoint_path = "./checkpoints/checkpoint.pt"
exports_path = "./exports"

Then, we'll load the base model checkpoint into memory from disk.

In [6]:
import torch

from model import UltraZoom

checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=True)

model = UltraZoom(**checkpoint["model_args"])

model.add_weight_norms()

model = torch.compile(model)

model.load_state_dict(checkpoint["model"])

model.remove_weight_norms()

model.eval()

print("Base checkpoint loaded successfully")

Base checkpoint loaded successfully


Now, let's export the model in HuggingFace format so that it can be used with the HuggingFace ecosystem.

In [7]:
from os import path

hf_path = path.join(exports_path, model_name)

model.save_pretrained(hf_path)

print(f"Model saved to {hf_path}")

Model saved to ./exports/UltraZoom-2X


Lastly, we'll login to HuggingFaceHub and upload the model under our account.

In [8]:
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub(model_name)

model.safetensors:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/andrewdalpino/UltraZoom-2X/commit/1ceda0a3f55ebdf70ca71713c66d2dc910c2f062', commit_message='Push model using huggingface_hub.', commit_description='', oid='1ceda0a3f55ebdf70ca71713c66d2dc910c2f062', pr_url=None, repo_url=RepoUrl('https://huggingface.co/andrewdalpino/UltraZoom-2X', endpoint='https://huggingface.co', repo_type='model', repo_id='andrewdalpino/UltraZoom-2X'), pr_revision=None, pr_num=None)